#### generate data to train CNN for low-wavenumber gradient prediction

In [1]:
from functions.F_plotting2 import *
from IPython.display import clear_output
import pickle
clear_output(wait=True)

/Users/pavelplotnitskii/Dropbox/Log_extrapolation/scripts/paper_reproduced


/Users/pavelplotnitskii/opt/anaconda3/envs/lw_mac/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Madagascar not found (called from functions.utils.loaders)! It is OK unless you want to generate data from scratch. Install m8r from ahay.org


### load data acquisition setup, which has been used in field data to generate synthetic data with these parameters

In [2]:
info_file=os.path.join('./data/acq_data_parameters_cgg.pkl')  # 80 sources
info_file=os.path.join('./data/acq_data_parameters_cgg_correct.pkl')
print(info_file)
with open(info_file,'rb') as input:
    acq_data=pickle.load(input)
log_dict=acq_data['log_dict']
log_loc=log_dict['loc']
log=log_dict['data']
idx = int(log_loc / 25)
vh = log_loc * np.ones_like(log)/1000

./data/acq_data_parameters_cgg_correct.pkl


### record data for cnn 

In [3]:
########################   parameters
launch_jobs  =0
record_tasks =1
run_fwi_flag =0
parallel_processing=1
flag_plotting=1
calculation_spacing=25
#################   FWI simulation parameters
pars={'data_mode':'cnn_16'}
pars.update({'current_data_type':'record_cnn_data'} )
#################   velocity model generator parameters
pars.update({'dx': calculation_spacing, 'dz': calculation_spacing,
    'out_shape':[496,150],  # grid size of generated model
    'dsrc':200, # source spacing
    'taper_shift':0,    #shift of the water taper
    'extend_model_x':False,
    'last_source_position':'nx',
    'computation_platform':'workstation',
    'gen_mode':'generator1',     # 'vlad' or 'oleg' option
    'initial_velocity_models_source':'generator',
    'root_denise':'',
    'data_gen_mode':'pseudo_field'
    })
################# seismic data filtering parameters
corner_frequency=5  # boundary of low frequencies
pars.update({'full_band':False})    # high-pass filter for seismic data 
pars.update({'corner_frequency':corner_frequency} )
pars.update({'delete_low_freqs':True} )
################# DENISE forward modelling parameters
pars.update({
        'NNODES': 1,'NPROCX':2,'NPROCY':1,      # small
        'ncores':20,'HOURS':24
        })

### Set up number of data samples to generate

In [4]:
res_folder='./data/gradients/'+pars['data_mode']
os.makedirs(res_folder,exist_ok=True)
processing_batch_size=10
sample_list=list(np.arange(0,2,1))
batches=[sample_list[x:x+processing_batch_size]   for x in range(0, len(sample_list), processing_batch_size)]
list_of_sbatch_files=[]

In [7]:
clear_output()
list_of_sbatch_files=[]
for batch in batches:
    for number in batch:
        velocity_model_name=number
        directory='model_'+str(number)
        results_folder = os.path.join(res_folder,directory)
        filename_model = os.path.join(results_folder,directory+'.hdf5')
        os.makedirs(results_folder,exist_ok=True)
        ########################   create folders and files to launch denise
        generate_data_for_batch_of_jobs(velocity_generator,denise_fwi,pars['gen_mode'],res_folder,calculation_spacing,pars,number)
        denise_fwi(filename_model,results_folder,os.getcwd(),calculation_spacing=calculation_spacing,pars=pars,mode='generate_task_files')
        os.system('export GEN='+os.path.join(results_folder,'fld'))
        os.system(f"mpirun -np {pars['ncores']} $DENISE/bin/denise $GEN/seis_forward.inp $GEN/seis_fwi.inp\n")
        # os.system(f"mpirun -np {pars['ncores']} $DENISE/bin/denise $GEN/seis_inversion.inp $GEN/seis_fwi.inp\n")
        ss=1
        ########################   create field data processing file
        # if pars['computation_platform']=='workstation':
        #     str1 = '#!/bin/bash\n'
        #     str1=str1+'export DENISE=./DENISE-Black-Edition\n'
        #     str1=str1+f"source ~/.bashrc\n"
        #     str1=str1+f"conda activate lw\n"
        #     str1=str1+f"which python\n"
        #     str1=str1+'export GEN='+os.path.join(results_folder,'fld')+'\n'
        #     #######
        #     str1 = str1 + f"python {os.path.join(results_folder,'data_generation_script.py')}\n"
        #     str1=str1+f"mpirun -np {pars['ncores']} $DENISE/bin/denise $GEN/seis_forward.inp $GEN/seis_fwi.inp\n"
        #     str1 = str1 + f"python {os.path.join(results_folder,'field_data_processing_script.py')}\n"
        #     str1=str1+f"mpirun -np {pars['ncores']} $DENISE/bin/denise $GEN/seis_inversion.inp $GEN/seis_fwi.inp\n"
        #     str1 = str1 + f"python {os.path.join(results_folder,'post_processing_script.py')}\n"
print('!!!!!!!!halas!!!!!!!!')

--------------------------------------------------------------------------
There are not enough slots available in the system to satisfy the 20
slots that were requested by the application:

  /bin/denise

Either request fewer slots for your application, or make more slots
available for use.

A "slot" is the Open MPI term for an allocatable unit where we can
launch a process.  The number of slots available are defined by the
environment in which Open MPI processes are run:

  1. Hostfile, via "slots=N" clauses (N defaults to number of
     processor cores if not provided)
  2. The --host command line parameter, via a ":N" suffix on the
     hostname (N defaults to 1 if not provided)
  3. Resource manager (e.g., SLURM, PBS/Torque, LSF, etc.)
  4. If none of a hostfile, the --host command line parameter, or an
     RM is present, Open MPI defaults to the number of processor cores

In all the above cases, if you want Open MPI to default to the number
of hardware threads instead of the num